# Campos de extracción de noticia

Titulo, contenido, autor, fecha, url, fotografía, tags, fecha_extraccion

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [16]:
from datetime import date, timedelta, datetime
today_date = datetime.today().strftime('%Y/%m/%d')
print(today_date, type(today_date))
d1 = date(2023, 1, 1)
d2 = date(2023, 10, 22)

# this will give you a list containing all of the dates
dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]

# for d in dd[:50]:
#     print(d)

2023/10/24 <class 'str'>


In [2]:
# Para extraer los artículos del día
url = "https://www.eldiario.net/portal/2023/01/01"
response = requests.get(url)
soup = BeautifulSoup(response.content, features="html.parser")

In [28]:
container = soup.body.find("div", class_="jeg_block_container")
block_articles = container.find_all("article", class_="jeg_post jeg_pl_md_2 format-standard")
contenido = dict()
contenido["url_base"] = []
contenido["download_date"] = []
contenido["title"] = []
contenido["url"] = []
contenido["fecha"] = []

url_base = "https://www.eldiario.net/portal/2023/01/01"
actual_date = url_base[-10:]

for article in block_articles:
    title = article.find("h3", class_="jeg_post_title")
    url = title.a["href"]
    
    if "PUBLICA TU AVISO" not in title.text:
        contenido["url_base"].append(url_base)
        contenido["download_date"].append(today_date)
        contenido["title"].append(title.text)
        contenido["url"].append(url)
        contenido["fecha"].append(actual_date)
df = pd.DataFrame(contenido)
df.to_csv("file.csv")
    

In [29]:

def extract_links(url, url_base, date, soup, contenido):
    container = soup.body.find("div", class_="jeg_block_container")
    block_articles = container.find_all("article", class_="jeg_post jeg_pl_md_2 format-standard")
    today = datetime.today().strftime('%Y/%m/%d')
    
    for article in block_articles:
        title = article.find("h3", class_="jeg_post_title")
        url = title.a["href"]
    
        if "PUBLICA TU AVISO" not in title.text:
            contenido["url_base"].append(url_base)
            contenido["download_date"].append(today_date)
            contenido["title"].append(title.text)
            contenido["url"].append(url)
            contenido["fecha"].append(actual_date)
    

In [30]:
def get_date(begin_date, end_date):
    d1 = datetime.strptime(begin_date, '%Y/%m/%d').date()
    d2 = datetime.strptime(end_date, '%Y/%m/%d').date()
    # this will give you a list containing all of the dates
    dd = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    
    return dd
    

In [36]:
def get_pages_links(start_url, soup):
    urls = []
    urls.append(start_url)
    page_info = soup.body.find("span", class_="page_info").text
    page = int(page_info.split()[-1])
    for i in range(2, page+1):
        url = start_url + "/page/" + str(i)
        urls.append(url)
    return urls
    # generate the urls
    

In [35]:
urls_list = get_pages_links(url_base, soup)
print(urls_list)

['https://www.eldiario.net/portal/2023/01/01', 'https://www.eldiario.net/portal/2023/01/01/page/2', 'https://www.eldiario.net/portal/2023/01/01/page/3', 'https://www.eldiario.net/portal/2023/01/01/page/4', 'https://www.eldiario.net/portal/2023/01/01/page/5', 'https://www.eldiario.net/portal/2023/01/01/page/6']


In [48]:
def get_articles_links_day(url_base):
    response = requests.get(url_base)
    
    if response.status_code != 200:
        print(f"Error in url {url_base}")
        return
        
    contenido = dict()
    contenido["url_base"] = []
    contenido["download_date"] = []
    contenido["title"] = []
    contenido["url"] = []
    contenido["fecha"] = []
    
    date = url_base[-10:]
    
    base_soup = BeautifulSoup(response.content, features="html.parser")
    pages_link = get_pages_links(url_base, base_soup)
    
    for url in pages_link:
        page_response = requests.get(url)
        if page_response.status_code != 200:
            print(f"Error in the url {url}")
            continue
        soup = BeautifulSoup(page_response.content, features="html.parser")
        
        extract_links(url, url_base, date, soup, contenido)

    df = pd.DataFrame(contenido)
    df.to_csv("2023-10-04.csv")

    return contenido["url"]
        


In [57]:
def get_article_content(article_url):
    article_page = requests.get(article_url)
    if article_page.status_code != 200:
        print(f"Error in url {article_url}")
        return
    article_name = article_url.split("/")[-2]
    print(article_name)
    try:
        soup_article = BeautifulSoup(article_page.text, features="html.parser")
        title = soup_article.body.find("h1", class_="jeg_post_title")
        print(title.text)
        date_container = soup_article.body.find("div", class_="jeg_meta_container")
        date = date_container.find("div", class_="jeg_meta_date")
        
        paragraphs = soup_article.body.find("div", class_="content-inner").find_all("p")
        abstract = ""
        for paragraph in paragraphs:
            abstract += paragraph.text
    except:
        print(f"error in {article_url}")

    with open(f"article_{article_name}.txt", "w") as f:
        content = title.text+"\n"+date.text+"\n"+abstract
        f.write(content)
        


In [49]:
link = get_articles_links_day("https://www.eldiario.net/portal/2023/01/01")

In [58]:
print(type(link))
for li in link:
    get_article_content(li)

<class 'list'>
explore-japon-en-una-exposicion
«Explore Japón» en una exposición
santa-cruz-recibe-2023-en-medio-de-represion-policial
Santa Cruz recibe 2023 en medio de represión policial
papa-emerito-fallecio-a-los-95-anos-de-edad
Papa emérito falleció a los 95 años de edad
la-prensa-es-blanco-del-abuso-de-uniformados
La prensa es blanco del abuso de uniformados
el-mundo-festeja-nuevo-ano-con-fuegos-artificiales
El mundo festeja nuevo año con fuegos artificiales
la-marraqueta-es-patrimonio-de-pacenos
La marraqueta es patrimonio de paceños
preocupa-a-hrw-proceso-contra-gobernador-camacho
Preocupa a HRW proceso  contra gobernador Camacho
la-paz-continua-registrando-mayor-cantidad-de-casos-de-feminicidios
La Paz continúa registrando mayor  cantidad de casos de feminicidios
camacho-recibio-el-nuevo-ano-en-chonchocoro
Camacho recibió  el nuevo año en  Chonchocoro
policia-descargo-gran-cantidad-de-gases-lacrimogenos
Policía descargó  gran cantidad de  gases lacrimógenos
marraqueta-es-patri